<h1> Лабороторна робота N1


In [47]:
import os
import urllib.request
from datetime import datetime
import pandas as pd

In [48]:
BASE_URL = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
YEARS = (1981, 2024)  # період
SAVE_DIR = "data"
os.makedirs(SAVE_DIR, exist_ok=True)
PROVINCES = {
    1: "Vinnytsia", 2: "Volyn", 3: "Dnipropetrovsk", 4: "Donetsk",
    5: "Zhytomyr", 6: "Zakarpattia", 7: "Zaporizhzhia", 8: "Ivano-Frankivsk",
    9: "Kyiv", 10: "Kirovohrad", 11: "Luhansk", 12: "Lviv",
    13: "Mykolaiv", 14: "Odesa", 15: "Poltava", 16: "Rivne",
    17: "Sumy", 18: "Ternopil", 19: "Kharkiv", 20: "Kherson",
    21: "Khmelnytskyi", 22: "Cherkasy", 23: "Chernivtsi",
    24: "Chernihiv", 25: "Crimea", 26: "Kyiv City", 27: "Sevastopol"
}


In [49]:
# === 1. Завантаження файлу ===
def download_vhi(province_id, province_name):
    """Завантаження VHI-файлу для області"""
    params = f"?country=UKR&provinceID={province_id}&year1={YEARS[0]}&year2={YEARS[1]}&type=Mean"
    url = BASE_URL + params
    filename = f"VHI_{province_id}_{province_name}.txt"
    filepath = os.path.join(SAVE_DIR, filename)

    if os.path.exists(filepath):
        print(f"[i] Файл для {province_name} уже є, пропуск...")
        return filepath

    try:
        print(f"[↓] Завантаження {province_name}...")
        urllib.request.urlretrieve(url, filepath)
        print(f"[✓] Збережено: {filepath}")
        return filepath
    except Exception as e:
        print(f"[!] Помилка завантаження {province_name}: {e}")
        return None

In [74]:
# === 2. Читання одного файлу ===
def read_vhi_file(filepath, province_id, province_name):
    import re
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()

        # Відкидаємо все до <pre>
        match = re.search(r"<pre>(.*)</pre>", text, re.DOTALL)
        if match:
            data_text = match.group(1)
        else:
            data_text = text

        # Розбиваємо на рядки
        lines = data_text.strip().splitlines()

        clean_lines = []
        for line in lines:
            line = line.strip().rstrip(",")  # видаляємо кінець рядка комою
            if not line:
                continue
            parts = line.split(",")
            if len(parts) >= 7:
                try:
                    year = int(parts[0])
                    clean_lines.append(parts[:7])
                except ValueError:
                    continue

        if not clean_lines:
            print(f"[!] Дані не знайдені у файлі {filepath}")
            return pd.DataFrame()

        df = pd.DataFrame(clean_lines, columns=["year","week","SMN","SMT","VCI","TCI","VHI"])

        df = df.astype({
            "year": int,
            "week": int,
            "SMN": float,
            "SMT": float,
            "VCI": float,
            "TCI": float,
            "VHI": float
        })

        df['province_id'] = province_id
        df['province_name'] = province_name

        return df

    except Exception as e:
        print(f"[!] Помилка обробки {province_name}: {e}")
        return pd.DataFrame()

In [51]:
# === 3. Об'єднання всіх областей ===
def load_all_vhi():
    all_dfs = []
    for pid, pname in PROVINCES.items():
        filepath = download_vhi(pid, pname)
        if filepath:
            df = read_vhi_file(filepath, pid, pname)
            if not df.empty:
                all_dfs.append(df)

    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    else:
        return pd.DataFrame()

In [52]:
def reindex_provinces_ukr(df):    
    # Сопоставление английских названий с украинскими
    english_to_ukr = {
        "Vinnytsia": "Вінницька", "Volyn": "Волинська", "Dnipropetrovsk": "Дніпропетровська",
        "Donetsk": "Донецька", "Zhytomyr": "Житомирська", "Zakarpattia": "Закарпатська",
        "Zaporizhzhia": "Запорізька", "Ivano-Frankivsk": "Івано-Франківська", "Kyiv": "Київська",
        "Kirovohrad": "Кіровоградська", "Luhansk": "Луганська", "Lviv": "Львівська",
        "Mykolaiv": "Миколаївська", "Odesa": "Одеська", "Poltava": "Полтавська",
        "Rivne": "Рівненська", "Sumy": "Сумська", "Ternopil": "Тернопільська",
        "Kharkiv": "Харківська", "Kherson": "Херсонська", "Khmelnytskyi": "Хмельницька",
        "Cherkasy": "Черкаська", "Chernivtsi": "Чернівецька", "Chernihiv": "Чернігівська",
        "Crimea": "Крим", "Kyiv City": "м.Київ", "Sevastopol": "Севастополь"
    }

    # Украинские названия по алфавиту
    ukr_provinces_sorted = [
        "Вінницька", "Волинська", "Дніпропетровська", "Донецька", "Житомирська",
        "Закарпатська", "Запорізька", "Івано-Франківська", "Київська", "Кіровоградська",
        "Луганська", "Львівська", "Миколаївська", "Одеська", "Полтавська",
        "Рівненська", "Сумська", "Тернопільська", "Харківська", "Херсонська",
        "Хмельницька", "Черкаська", "Чернівецька", "Чернігівська", "Крим",
        "м.Київ", "Севастополь"
    ]
    ukr_index_map = {name: idx + 1 for idx, name in enumerate(ukr_provinces_sorted)}

    # Копируем DataFrame, чтобы не менять оригинал
    df = df.copy()
    df['province_id'] = df['province_name'].map(english_to_ukr).map(ukr_index_map)
    df['province_name'] = df['province_name'].map(english_to_ukr)
    # Преобразуем английские имена в украинские, затем присваиваем ID по алфавиту
   
    #df['province_name'] = df['province_name'].map(english_to_ukr)
    # Проверка: если остались NaN
    missing = df[df['province_id'].isna()]['province_name'].unique()
    if len(missing) > 0:
        print("[!] Не найденные названия для переиндексации:", missing)

    return df

In [75]:
# === Основний блок ===
if __name__ == "__main__":
    df_all = load_all_vhi()
    for pid, pname in PROVINCES.items():
        filepath = os.path.join(SAVE_DIR, f"VHI_{pid}_{pname}.txt")
        df = read_vhi_file(filepath, pid, pname)
        print(pid, pname, len(df))
    if not df_all.empty:  
        df_all = reindex_provinces_ukr(df_all)
        df_all.to_csv("all_vhi_cleaned.csv", index=False)        
        print("[✓] Дані збережено у all_vhi_cleaned.csv")
        print(df_all.head())
        # якщо треба подивитись сортування:
        print(df_all.sort_values('province_id'))
    else:
        print("[!] Дані не були завантажені")

[i] Файл для Vinnytsia уже є, пропуск...
[i] Файл для Volyn уже є, пропуск...
[i] Файл для Dnipropetrovsk уже є, пропуск...
[i] Файл для Donetsk уже є, пропуск...
[i] Файл для Zhytomyr уже є, пропуск...
[i] Файл для Zakarpattia уже є, пропуск...
[i] Файл для Zaporizhzhia уже є, пропуск...
[i] Файл для Ivano-Frankivsk уже є, пропуск...
[i] Файл для Kyiv уже є, пропуск...
[i] Файл для Kirovohrad уже є, пропуск...
[i] Файл для Luhansk уже є, пропуск...
[i] Файл для Lviv уже є, пропуск...
[i] Файл для Mykolaiv уже є, пропуск...
[i] Файл для Odesa уже є, пропуск...
[i] Файл для Poltava уже є, пропуск...
[i] Файл для Rivne уже є, пропуск...
[i] Файл для Sumy уже є, пропуск...
[i] Файл для Ternopil уже є, пропуск...
[i] Файл для Kharkiv уже є, пропуск...
[i] Файл для Kherson уже є, пропуск...
[i] Файл для Khmelnytskyi уже є, пропуск...
[i] Файл для Cherkasy уже є, пропуск...
[i] Файл для Chernivtsi уже є, пропуск...
[i] Файл для Chernihiv уже є, пропуск...
[i] Файл для Crimea уже є, пропуск..

In [12]:
def vhi_by_region_year(df, province_name, year):
    """
    Повертає ряд VHI для конкретної області за вказаний рік.
    
    df : pd.DataFrame
    province_name : str - назва області українською
    year : int - рік
    """
    filtered = df[(df['province_name'] == province_name) & (df['year'] == year)]
    return filtered[['year', 'week', 'VHI', 'province_name']]

In [13]:
def vhi_by_regions_years(df, provinces, year_start, year_end):
    """
    Повертає ряд VHI для кількох областей за вказаний діапазон років.
    
    provinces : list[str] - список областей українською
    year_start : int
    year_end : int
    """
    filtered = df[
        (df['province_name'].isin(provinces)) &
        (df['year'] >= year_start) &
        (df['year'] <= year_end)
    ]
    return filtered[['year', 'week', 'VHI', 'province_name']]

In [21]:
def vhi_stats(df, provinces, years):
    """
    Обчислює min, max, mean, median VHI для вказаних областей та років.
    
    provinces : list[str] - список областей
    years : list[int] - список років
    """
    filtered = df[
        (df['province_name'].isin(provinces)) &
        (df['year'].isin(years))
    ]
    
    stats = filtered.groupby('province_name')['VHI'].agg(['min','max','mean','median']).reset_index()
    return stats

In [76]:
# ряд VHI для Києва за 2011
print(vhi_by_region_year(df_all, "Київська", 2011).head())

# ряд VHI для Києва та Львова за 2010-2012
print(vhi_by_regions_years(df_all, ["Київська","Львівська"], 2010, 2012).head())

# статистика для Києва та Львова за 2010-2012
print(vhi_stats(df_all, ["Київська","Львівська","Вінницька"], [2010,2011,2012]))



       year  week    VHI province_name
19396  2011     1  39.77      Київська
19397  2011     2  38.20      Київська
19398  2011     3  38.36      Київська
19399  2011     4  39.36      Київська
19400  2011     5  41.32      Київська
       year  week    VHI province_name
19344  2010     1  50.19      Київська
19345  2010     2  48.41      Київська
19346  2010     3  47.55      Київська
19347  2010     4  47.77      Київська
19348  2010     5  48.36      Київська
  province_name    min    max       mean  median
0     Вінницька  28.04  72.75  44.589744  42.680
1      Київська  19.42  69.48  39.700577  39.105
2     Львівська  26.81  62.09  43.638718  44.460


In [77]:
# Читання з урахуванням розділювача і пропусків
file_path = "household_power_consumption.txt"
df = pd.read_csv(file_path, sep=";", na_values="?", low_memory=False)
print(df.info())
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'household_power_consumption.txt'